#####  *Python libraries used for this notebook project* #####

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy.optimize import linprog

<h1><center> PROPOSAL FOR THE IMPROVEMENT OF A MATHEMATICAL ALGORITHM </center></h1>
<h2><center> FRAX® FOR MEDICAL DIAGNOSIS </center></h2>

## Abstract
The algorithm of the Fracture Risk Assessment Tool (FRAX) is a web application to estimate the 10-year probability of suffering a hip fracture or any other osteoporotic fracture based on a series of clinical risk factors for individuals between the ages of 40 and 90 years old.

With the purpose to improve the accuracy of the algorithm, a specific risk factor is added to the initial information: a chronological series of radiographs of the patient. An additional improved algorithm is added to automatically analyze such radiographs to calculate a more accurate risk factor.

## Table of contents

1. [Introduction](#1.-Introduction)
2. [Fracture Risk Assesment Tool](#2.-Fracture-Risk-Assesment-Tool) 
     * 2.1. [What is FRAX?](#2.1-What-is-FRAX?) 
     * 2.2. [Risk Factors](#2.2-Risk-Factors)
3. [Improvement of the FRAX web application](#3.-Improvement-of-the-FRAX-web-application)
     * 3.1. [Objective](#3.1-Objective)
     * 3.2. [X-rays, a specific risk factor](#3.2-X-rays,-a-specific-risk-factor)
     * 3.3. [Earth mover's distance](#3.3-Earth-mover's-distance)
     * 3.4. [Programming code](#3.4-Programming-code)
     * 3.5. [Implementation of the proposal with real case scenarios](#3.5-Implementation-of-the-proposal-with-real-case-scenarios)
4. [Last conclutions](#4.-Last-conclutions)
5. [Bibliography](#5.-Bibliography)
 


### 1. Introduction

This chapter introduces the different backgrounds and motivations to develop this final project.

**Bone fractures** are a major public health problem. Hip fracture is one of the most important causes of morbidity and mortality in elderly patients and is a challenge for health systems worldwide, both due to its frequency and its high economic cost. The mortality after the hip fracture during the first year after the facture is around 19%.**[1]** The epidemiological data varies between countries, but it is globally estimated that hip fractures will affect around 18% of women and 6% of men.
Hip fractures are a major public health problem, the global number is expected to increase from 1.26 million in 1990 to 4.5 million by the year 2050. For example, only for U.S. Hip fracture represents a substantial economic burden to health-care system, accounting for $5.96 billion per year. This cost is expected to triplicate for the next 20 years.**[2]**

Early identification of patients at higher risk of suffering a fracture is essential to prevent its occurrence. 
To do this, different tools are designed based on mathematical algorithms to assess the risk of fracture that feed different patient data. For example, *FRAX®*, *QFracture®*, Canadian Osteoporosis Society, the Nguyen algorithm and the Dutch algorithm.

**Osteoporosis** weakens bones to the point that they can break easily. It is called a “silent disease” because people who develop it may not notice any changes until a bone breaks — usually a bone in the hip, spine, or wrist. While people of all races and ethnic groups can develop osteoporosis, certain groups are more likely to develop the disease. 
It is a disease linked to aging, which affects about one in five women and one in twenty men, over age of 50.

**Bone Mineral Density(BMD)** is used as an **indirect indicator** of osteoporosis and fracture risk. BDM is the amount of bone mineral in bone tissue. The concept is of mass of mineral per volume of bone (relating to density in the physics sense), although clinically it is measured by proxy according to optical density per square centimetre of bone surface upon imaging. The most commonly used BMD test is called a **central dual-energy X-ray absorptiometry**(**DXA**). Bone mineral density could help prevent fractures, allowing the diagnosis of osteoporosis before a fracture event occurs.

However, the imperfect capture of risk with BMD alone poses several problems for the clinical assessment of fracture risk.
The performance characteristics of the test are less than optimal in terms of the trade-off between sensitivity and specificity. 
The osteoporotic fractures affect a substantial minority of the population, so based only on BDM detection, sensivity is low. 
For example, the annual incidence of hip fracture increases approximately 30 times between the ages of 50 and 90, however considering only the bone mineral density basis, studies expect an increase of only 4 times. **[3]**
Another big disadvantage of BMD is its high cost. There are many countries that currently do not have such a device widely distributed.

Due to the difficulties presented by the BMD determination test itself, other tools emerge considering more independent risk factors.

The main objective of this work is try to improve the *FRAX* tool, increasing its probability accuracy through an algorithm that analyzes a series of additional radiographs of patients.

### 2. Fracture Risk Assesment Tool

#### 2.1 What is FRAX?

The Fracture Risk Assesment Tool (*FRAX*) is a mathematical algorithm, which was developed by Kanis and others at the University of Sheffield with the sponsorship of **World Health Organization (WHO)**, with the purpose to assess the risk of suffering a fracture in patients. 
*FRAX* is an *on-line* tool which calculates the risk of fracture from a number of clinical risk factors, whether bone mineral density is known or not. It is probably the most internationally recognized method for determining fracture risk, thanks to the quality of the data used on its methodology and calculations. 

This *FRAX* model has been validated in 11 study cohorts across the world. The application computes the probability of both hip fracture and the most important osteoporotic fractures at a 10-year span (spinal, forearm, shoulder, or hip clinic).

*FRAX*  is not an infallible tool and it does have its **limitations**:

- It should be noted that the Body Mineral Density on its own is not an accurate method for predicting osteoporotic fractures, but *FRAX* works better if the BMD is known. 
- Other example could be the **Body Mass Index** $ BMI = \frac{mass(kg)} {height(m)^2} $ which may be altered in patients with high muscular development (BMI increases, the risk may be overestimated) or by decreased height in patients due to vertebral fractures. In these patients, the risk may be underestimated (worst possible scenario). 

#### 2.2 Risk Factors


*FRAX* uses several risk factors to calculate the individual´s probability of hip fracture or osteoporotic fracture.
In the process of determining an individual's probability of fracture, the *FRAX* tool takes into account different risk indicators.

1. Bone Mineral Density (BMD)

Bone mineral density, as a risk factor of the *FRAX* algorithm, is evaluated in the *femoral neck* by DXA, but
there are a few cohort studies where DXA was used on the *distal forearm*. As mentioned in the previous section, *FRAX* may or may not count on the BMD value. To measure this variable the **T-Score** is used, according to the World Health Organization. This score indicates the amount in **standard deviations(SD)** that the subject varies from the young adult mean.
The following table details the DS of T-Score criteria for the diagnosis of osteoporosis according to the WHO.

|T-Score                                      |  Category            |
|:---                                         |:---                  |
|T-Score > -1 SD                              | Normal               |
|-2,5 SD < T-Score < -1 SD                    | Osteopenia           |
|T-Score < -2,5 SD                            | Osteoporosis         |
|T-Score < -2,5 SD + one or more fractures    | Severe osteoporosis  |

<h6><center> Table 1. Diagnosis criteria for osteoporosis according to T-Score </center></h6>

As previously mentioned, having BMD alone is insufficient to identify all individuals under risk of fracture, since these fractures affect a minority of the population (low sensitivity). On the other hand, it should be noted that obtaining a T-Score > 1,5 SD does not guarantee not suffering a fracture. Therefore, to better estimate the risk there is a need to take into consideration a wider range of risk factors.

2. Age

The model accepts ages between 40 and 90 years. If ages below or above are entered, the programme will compute probabilities at 40 and 90 years old, respectively. The increase in risk with increasing age can vary up to 4 times when compared to BMD alone**[3]**.

3. Sex

It is a binary factor, male or female. Men are less susceptible to osteoporosis than women, especially in the years when estrogen levels decrease, that is, the period after menopause.

4. Weight

This should be entered in kg.

5. Height

This should be entered in cm.

6. BMI

The patient does not have to calculate it manually, the algorithm works automatically. It is expressed in 𝑘𝑔/𝑚2. At one end a low BMI leads to a higher probability of fracture, and in the other, obesity is not a protective measure against fracture in postmenopausal women and therefore it could increases the risk of fracture.**[4]**

6. Previous fracture

It is a binary factor, yes or not. A previous fracture denotes more accurately a previous fracture in adult life occurring spontaneously, or a fracture arising from trauma which, in a healthy individual, would not have resulted in a fracture.


7. Parent fractured hip

It is binary factor, yes or not. This enquires for a history of hip fracture in the patient's mother or father. 
A parental history of fracture was associated with a modest but significantly increased risk of any fracture, osteoporotic fracture and hip fracture in men and women combined. **[5]**

8. Current smoking

It is a binary factor, yes or not depending on whether the patient currently smokes tobacco. It is known that smoking is a dose-dependent risk factor, so the risk increases with increasing doses **[6]**. However, the amount of tobacco consumed is not taken into account in the calculations since the model is based on information from cohort studies where such information is not available. 

9. Glucocorticoids

It is abinary factor, yes or not. As in the case of tobacco, the dose consumed is not taken into account in the calculations. However, there is a dose-risk relationship. **[7], [8]**

10. Rheumatoid arthritis

It is a binary factor, yes or not. A factor that has importance on the risk of osteoporotic fracture. Increased prevalence of osteoporosis 2 times even in populations with younger ranges than FRAX. **[9]** Diagnosed rheumatoid arthritis carries a higher risk of fracture than indicated by BMD alone. **[8]**


11. Secondary osteoporosis

It is a binary factor, yes or not. Secondary osteoporosis include disorder strongly associated with osteoporosis: diabetes type I (insulin dependent), osteogenesis imperfecta in adults, untreated long-standing hyperthyroidism, hypogonadism or premature menopause (<45 years), chronic malnutrition, or malabsorption and chronic liver disease.


12. Alcohol

It is a binary factor, yes if the patient takes 3 or more units of alcohol daily. A unit of alcohol varies slightly in different countries from 8-10g of alcohol. This is equivalent to a standard glass of beer (285ml), a single measure of spirits (30ml), a medium-sized glass of wine (120ml), or 1 measure of an aperitif (60ml). The amount of alcohol consumed is not taken into account. However, the risk steadily increased with higher intake of alcohol. **[10]**




### 3. Improvement of the *FRAX* web application

#### 3.1 Objective

This project has the purpose to improve the *FRAX* current model, increasing the accuracy of the fracture probability by adding a specific risk factor to the initial information of the patient: a chronological series of radiographs.

#### 3.2 X-rays, a specific risk factor

To improve the application of *FRAX* an additional factor is added, a series of radiographs, to obtain a more accurate fracture probability of hip or osteoporotic fractures.
<img src="https://raw.githubusercontent.com/mstoychev/Math-Concepts-for-Developers/main/8.%20Final%20Project/woman_0.jpeg?token=GHSAT0AAAAAACBQP36W4W3BH2ZF36AQWDMKZDKIF5A" alt="Healthy woman radiography" style="width:200px;height:250px"/> 
<h6><center>Radiography of a 53-year-old woman</center></h6>

Conventional radiographs are not the most appropriate method for the early diagnosis of osteoporosis. This is due to the fact that in order to detect deformations in the image there must be at least a minimum between 30% to 50% loss of bone mass **[11]**. However, radiographs are a widespread method and very useful technique for the detection of osteoporotic fractures. The decrease in bone mineral density is reflected on X-ray images as a darkening area of the bone.
<img src="https://raw.githubusercontent.com/mstoychev/Math-Concepts-for-Developers/main/8.%20Final%20Project/woman_arrow.jpg?token=GHSAT0AAAAAACBQP36XNZBPZEXJD6X57IFAZDKIGOQ" alt="Osteoporotic woman arrow" style="width:200px;height:250px"/> 
<h6><center>Radiography with bone degradation</center></h6>

#### 3.3 Earth mover's distance

In computer science, the **Earth mover's distance (EMD)** is a distance-like measure of dissimilarity between two frequency distributions, densities, or measures over a region $D$. For probability distributions and normalized histograms, it reduces to the **Wasserstein metric**. It is a measure of the distance between two probability distributions in a given region.

Multidimensional distributions are used in the description of features of an image. Due to this, the EMD is especially useful in measuring the similarity of two images with respect to their color and texture.

To optimize processing time, the original distribution is converted to compressed distribution. In practice with images, compressing a multidimensional distribution is achieved using histograms.
A histogram of a grayscale image is a graphical representation of the distribution of tones in the image. It is a plot with pixel values ranging from 0 to 255 in X-axis and corresponding number of pixels in the image on Y-axis.

The distance between two grayscale images can be calculated using the Earth mover´s distance. Intuitively, if each distribution is viewed as a unit amount of earth (soil) piled on $D$, the metric is the minimum amount of work of turning one pile into the other, which is assumed to be the amount of earth that needs to be moved times the mean distance it has to be moved.

The EMD is based on solving a transportation problem using *linear programming*. Let be a set of points in $\mathbb{R}^d$ with $d$ the dimension. These points are grouped in order to be represented in terms of clusters. Thus, each cluster is a single point in $\mathbb{R}^d$ and the weight of the cluster is decided by the fraction of the distribution present in that cluster. This representation of a distribution by a set of clusters is called the *Signature*. So a histogram can be interpreted as a signature.

The objective is transforming one signature $P$ to another $Q$ with minimum amount of work. Assume that the signatures $P$ and $Q$ have $m$ and $n$ clusters respectively. 

$ P = \{(p_1, w_{p1}),(p_2, w_{p2}), ... , (p_m, w_{pm})\} $

$ Q = \{(q_1, w_{q1}),(q_2, w_{q2}), ... , (q_m, w_{qn})\} $

where $p_i$ and $q_j$ are the representatives of the clusters and  $w_{pi}$ and $w_{qj}$ their weights. It must always be true that $w_{pi} > 0$ and $w_{qj} > 0$ .


Let also $ D = \begin{bmatrix} d_{ij}\end{bmatrix} $ be the ground distance matrix between clusters $p_i$ and $p_j$.

The idea is to find a flow $ F = \begin{bmatrix} f_{ij}\end{bmatrix} $, with $f_{ij}$ the flow between $p_i$ and $q_j$ that minimizes the overall cost:

$\min_\limits{F} (P, Q) = \sum_{i=1}^{m} \sum_{j=1}^{n} f_{ij} d_{ij}$ 

subject to the next constraints:

$ f_{ij} \ge 0, 1\le i \le m,  1\le j \le n $

$ \sum_{j=1}^{n} f_{ij} \le w_{pi}$ , $ 1 \le i \le m $

$ \sum_{i=1}^{m} f_{ij} \le w_{qj}$ , $ 1 \le j \le n $

$ \sum_{i=1}^{m} \sum_{j=1}^{n} f_{ij} = \min\left\lbrace \sum_{i=1}^{m} w_{pi}, \sum_{j=1}^{n} w_{qj} \right\rbrace $


The optimal flow $F$ is found by solving the linear optimization problem. Earth mover´s distance is defined as the resulting work normalized by the total flow. That is to say,

**EMD (P,Q)** = $\frac{\min_\limits{F} (P, Q)}{\sum_{i=1}^{m} \sum_{j=1}^{n} f_{ij}}$

The EMD is applied to the signatures that subsume the histograms. It is computed, returning as a result the flow that minimizes the cost function and the value of this flow, which is interpreted as the distance between two images.

#### 3.4 Programming code

In [2]:
def euclidean_distance(f1, f2):
    '''
    f1 is the first representative p_i
    f1 is the second representative q_i
    This function calculates Euclidean Distance between the representatives.
    
    '''
    d = np.linalg.norm(f1 - f2, 2)
    return d


def distance_matrix(f1, f2):
    '''
    f1 is the first representative p_i
    f1 is the second representative q_i
    This function calculates the distance matrix D, where Euclidean Distance is being used.
    
    '''
    m = len(f1)
    n = len(f2)
    D = np.zeros((m, n))

    for i in range(m):
        for j in range(n):
            D[i, j] = euclidean_distance(f1[i], f2[j])
    
    D = D.T.flatten()  # Transpose the column vector and flatten to a list
    return D


def calculate_emd(f1, f2, w1, w2):
    # Matrix distance D to the floor 
    D = distance_matrix(f1, f2)

    m = len(f1)
    n = len(f2)

    ################################################### CONSTRAINS ############################################################
    # Create empty matrix with dimentions m x m*n
    A1 = np.zeros((m, m * n))
    A2 = np.zeros((n, m * n))

    for i in range(m):
        for j in range(n):
            k = j + i * n  # variable k represents the column index in the matrices
            A1[i, k] = 1  # value of 1 is chosen for convenient as being used in resolving linear programming problem
            A2[j, k] = 1
    
    #Stacking vertically (row-wise) A1 and A2, which results in matrix A with dimensions (m + n) x (m * n)
    A = np.vstack((A1, A2))
    
    #b performs a horizontal concatenation of the arrays w1 and w2. The resulting array has a shape of (1, m + n)
    b = np.concatenate((w1, w2))

    # Constrains to equality
    Aeq = np.ones((m + n, m * n))
    beq = np.ones(m + n) * min(np.sum(w1), np.sum(w2))

    # Lower limit
    lb = np.zeros(m * n)

    # linprog resolves the linear programming problem
    resolution = linprog(D, A_ub=A, b_ub=b, A_eq=Aeq, b_eq=beq, bounds=[(l, None) for l in lb], method='highs')
    EMD = resolution.fun / np.sum(resolution.x)  #EMD (objective function value in linear programming)
    return EMD


def emd_images(img1, img2):
    # Histograms (Signatures)
    nbins = 100
    channel_a = cv2.calcHist([img1], [0], None, [nbins], [0, 256])
    channel_b = cv2.calcHist([img2], [0], None, [nbins], [0, 256])

    # Representatives of the clusters
    ha, _ = np.histogram(img1, bins=nbins)
    hb, _ = np.histogram(img2, bins=nbins)

    # Convert representatives to 2D arrays
    f1 = np.column_stack((ha, np.zeros_like(ha)))
    f2 = np.column_stack((hb, np.zeros_like(hb)))

    # Weights
    w1 = channel_a.flatten() / np.sum(channel_a)
    w2 = channel_b.flatten() / np.sum(channel_b)

    # EMD
    EMD = calculate_emd(f1, f2, w1, w2)
    return EMD

#### 3.5 Implementation of the proposal with real case scenarios ####
In this section the risk of two patients will be estimated. The available data are the risk factors for the *FRAX* tool and a series of additional radiographs.
The first patient with ID 4532 is a 59 year old man, 99,50 kg weight and 181 cm height. Regarding the generic risk factors, it is known that he is diabetic, he has suffered a previous fracture and his parents have also suffered hip fracture. He does not consume glucocorticoids and tobacco, but he consumes alcohol. The bone mineral density test is available with 𝑇−𝑠𝑐𝑜𝑟𝑒=−1,2.

<img src="https://raw.githubusercontent.com/mstoychev/Math-Concepts-for-Developers/main/8.%20Final%20Project/FRAX_man.JPG?token=GHSAT0AAAAAACBQP36XGYCTVPCEBOKMWBKCZDJ7RCQ" alt="Frax questionarie for man" style="width:650px;height:400px"/> 
<h6><center>Frax questionnaire for patient ID 4532 </center></h6>

Patient ID 4532. The patient feels a lack of hip mobility on the left side. For this reason, her doctor schedules a series of x-rays with a period of 6 months difference between each of them. The results are the following:

<img src="https://raw.githubusercontent.com/mstoychev/Math-Concepts-for-Developers/main/8.%20Final%20Project/man_summary.JPG?token=GHSAT0AAAAAACBQP36XACK3I5DIOYVUUTTKZDJ7RTA" alt="Radiography summary man" style="width:650px;height:400px"/> 
<h6><center>Series of radiographs with 6 months time difference, ID 4532 </center></h6>

The second patient with ID 2869 is a 52 year old woman, 47.32 kg weight and 157 cm height. Regarding the generic risk factors, it is known that she has not suffered any previous fractures but one of her parents suffered a hip fracture. The patient suffers from rheumatoid arthritis and consumes glucocorticoids. However, she does not consumes alcohol and tobacco. The bone mineral density test is available with 𝑇−𝑠𝑐𝑜𝑟𝑒=−2.1.

<img src="https://raw.githubusercontent.com/mstoychev/Math-Concepts-for-Developers/main/8.%20Final%20Project/FRAX_woman.jpg?token=GHSAT0AAAAAACBQP36XA7EVODDNMTQGZZPQZDJ7M4Q" alt="Frax questionarie for woman" style="width:650px;height:400px"/> 
<h6><center>Frax questionnaire for patient ID 2869 </center></h6>

Patient ID 2869. The patient suffers from pain on the right side of the hip. For this reason, her doctor schedules a series of x-rays with a period of 6 months difference between each of them. The results are the following:
<img src="https://raw.githubusercontent.com/mstoychev/Math-Concepts-for-Developers/main/8.%20Final%20Project/woman_summary.JPG?token=GHSAT0AAAAAACBQP36WWURN2LHOXN7RUNJQZDJ7SQQ" alt=" woman summary" style="width:650px;height:450px"/> 
<h6><center>Series of radiographs with 6 months time difference, ID 2869 </center></h6>

In [3]:
# create an empty data store diccionary
data_store = {}

# create variable for both patient
patient_1 = "ID_4532"
patient_2 = "ID_2869"

# store FRAX´s hip fracture risk in the dictionary
frax_risk_1, frax_risk_2  = 0.026, 0.025
data_store[patient_1] = [frax_risk_1]
data_store[patient_2] = [frax_risk_2]


# Patient ID 4532
img1 = cv2.imread("man_0.jpg", cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread("man_1.jpg", cv2.IMREAD_GRAYSCALE)
img3 = cv2.imread("man_2.jpg", cv2.IMREAD_GRAYSCALE)
img4 = cv2.imread("man_3.jpg", cv2.IMREAD_GRAYSCALE)


distance = emd_images(img1, img2)
print(f"Patient {patient_1}, Earth Mover's Distance 0-12 months: {distance}")

distance = emd_images(img1, img3)
print(f"Patient {patient_1}, Earth Mover's Distance 0-18 months: {distance}")

distance = emd_images(img1, img4)
print(f"Patient {patient_1}, Earth Mover's Distance 0-24 months: {distance}")

Patient ID_4532, Earth Mover's Distance 0-12 months: 439.39234016021965
Patient ID_4532, Earth Mover's Distance 0-18 months: 654.2887280578416
Patient ID_4532, Earth Mover's Distance 0-24 months: 702.6868378580954


In [4]:
# Patient ID 2869
img1 = cv2.imread("woman_0.jpg", cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread("woman_1.jpg", cv2.IMREAD_GRAYSCALE)
img3 = cv2.imread("woman_2.jpg", cv2.IMREAD_GRAYSCALE)
img4 = cv2.imread("woman_3.jpg", cv2.IMREAD_GRAYSCALE)
img5 = cv2.imread("woman_4.jpg", cv2.IMREAD_GRAYSCALE)

distance = emd_images(img1, img2)
print(f"Patient {patient_2}, Earth Mover's Distance 0-6 months: {distance}")

distance = emd_images(img1, img3)
print(f"Patient {patient_2}, Earth Mover's Distance 0-12 months: {distance}")

distance = emd_images(img1, img4)
print(f"Patient {patient_2}, Earth Mover's Distance 0-18 months: {distance}")

distance = emd_images(img1, img5)
print(f"Patient {patient_2}, Earth Mover's Distance 0-24 months: {distance}")

Patient ID_2869, Earth Mover's Distance 0-6 months: 70.43604509478669
Patient ID_2869, Earth Mover's Distance 0-12 months: 74.82613242644113
Patient ID_2869, Earth Mover's Distance 0-18 months: 114.87297730968152
Patient ID_2869, Earth Mover's Distance 0-24 months: 150.17002954437982


|Temporary X-Ray  |  EMD result: ID_4532   |  EMD result: ID_2869   |  
|-----------------|------------------------|------------------------|
|0-6 months       |          --            |           70.43        |
|0-12 months      |          439.39        |           74.82        |
|0-18 months      |          654.28        |           114.87       |
|0-24 months      |          702.68        |           150.17       |

<h6><center> Table 2. EMD calculated for both patients </center></h6>


In both patients we can observe a worsening of bone tissue during a period of approximately two years. This evolution of the patient must be calculated as a probability of fracture and this must be included in the probability already calculated with *FRAX*.

The following tables summarize the calculations of the probability of hip fracture based on the EMD. The detailed explanation is in the file Risk Fracture Calculation.xlsx

|Patient ID_4532             |  6 months   |  12 months   |  18 months   |  24 months  |    TOTAL    |   
|-----------------|------------------------|--------------|--------------|-------------|-------------|
|Days after the first X-ray  |     --      |     384      |     201      |    14       |     599     |
|Weights of the days         |     --      |    1.559     |     2.980    |    42.78    |     47.326  |
|Ponderation of days         |     --      |    0.033     |     0.062    |    0.090    |             |
|                            |                                                                       |
|EMD                         |     --      |    439.39    |     654.28   |    702.68   |   1796.36   |
|EMD weigth                  |     --      |    0.24      |     0.36     |    0.39     |             |
|                            |                                                                       |
|Probability                 |     --      |   0.008      |     0.019    |    0.394    |             |
|**Fracture hip risk**       |             |              |              |             | **12.82%**  |

<h6><center> Table 3. Patient ID_4532 - Hip fracture probability calculated from X-rays </center></h6>


|Patient ID_2869             |  6 months   |  12 months   |  18 months   |  24 months  |    TOTAL    |   
|-----------------|------------------------|--------------|--------------|-------------|-------------|
|Days after the first X-ray  |     568     |     381      |     202      |    18       |     1169    |
|Weights of the days         |     1.054   |    1.572     |     2.965    |    33.27    |     38.869  |
|Ponderation of days         |     0.027   |    0.040     |     0.076    |    0.856    |             |
|                            |                                                                       |
|EMD                         |     70.43   |    74.82     |     114.87   |    150.17   |    410.35   |
|EMD weigth                  |     0.17    |    0.18      |     0.27     |    0.36     |             |
|                            |                                                                       |
|Probability                 |     0.004   |    0.008     |     0.022    |    0.280    |             |
|**Fracture hip risk**       |             |              |              |             | **8.67%**   |

<h6><center> Table 4. Patient ID_2869 - Hip fracture probability calculated from X-rays </center></h6>

Now with this results, **two events are defined**:
* $A$:*Ten year probability of hip fracture due to generic risk factors (from FRAX questionnaire)*
* $B$: *Risk of hip fracture due to quality of the bone (from serie of radiography)*

where for the patient ID 4532, $P(A)$ = 0.026 and $P(B)$ = 0.14.

Also for the patient ID 2869, $P(A)$ = 0.025 and $P(B)$ = 0.079.


Assuming that both events are **independent** of each other and applying the definitions, the probability of the union of two events can be found with the addition rule:

$P(A \cup B) = P(A) + P(B) - P(A \cap B) $

$P(A \cup B) = P(A) + P(B) - P(A)*P(B) $

In [5]:
x_ray_prob1 = 0.128214418
x_ray_prob2 = 0.086683367

data_store[patient_1].append(x_ray_prob1)
data_store[patient_2].append(x_ray_prob2)

for k,v in data_store.items():
    p_a, p_b = v[0], v[1]
    probability = p_a + p_b - (p_a*p_b)
    print(f"The patient {k} has {(probability*100):.1f}% hip fracture risk")

The patient ID_4532 has 15.1% hip fracture risk
The patient ID_2869 has 11.0% hip fracture risk


Only from the generic risk factors used with the *FRAX* algorithm, the risk of a hip fracture obtained for the next 10 years for the patient ID_4532 is 2,6% and for the patient_2869 is 2,5%. The new proposed model increases the risk of fracture in the patients to 15.1% and 11.0% respectively. 

Using such an important factor as the clear deterioration of the femur bone over two years in the calculation of the probability of a hip fracture resulted in a much more accurate model, as expected.

This new model was tested in two specific cases with a man and a woman, but it can be used in all other case scenarios. The comparison of X-ray images however must be between a series of images of the same bone. In this case, the hip was used because it is both the most widely used method in science and the one also used mainly by *FRAX* in the studies on which it is based for the creation of its mathematical algorithm. However, some *FRAX* studies use the distal forearm bone. The method should be tested, but in principle there should be no restriction for its correct operation.

### 4. Last conclutions

The objective of the *FRAX* calculation tool is to determine the 10-year probability of experiencing an osteoporotic fracture or a hip fracture for individuals between 40 and 90 years old. It is concluded that *FRAX* is a fairly accurate application being based on international cohorts and externally validated provides high-quality data, resulting in reliable fracture probabilities.

However, the questionnaire has limitations and could be subject to improvement. This project proposes an improvement introducing a new specific risk factor which is a series of patient X-ray images. 

The programming language used in this model was Python. There are functions already created in the cv2 library, such as the Wasserstein distance, which is an approximation of the EMD. 
In this project, the decision was made to use the mathematical definition of the Earth Movers Distance for the creation of the improvement algorithm because this project is focused on mathematics.

In addition to the X-ray factor, there are other future lines of research to obtain a more precise fracture probability. For example, the inclusion of more generic factors in the questionnaire that influence the estimates of the risk functions for fracture. Variables such as the patient's physical exercise, nutrition or whether they consume calcium and vitamin D supplements or sun exposure could be taken into account.

Practicing sports in moderation is also a prevention method, especially when working with weights, since it helps strengthen bone mass and prevents bone loss, especially notably after the age of 40, and therefore reduces the risk of fracture.**[12]** However, excessive sport and risky sports can increase the risk of fracture.

A varied diet based on whole vegetables is key to preventing fractures. However, there are different controversies about the consumption of calcium. For example, in the UK the adult recommendation is 700mg per day, but in the US it is 1200mg per day. These discrepancies are a clear sign of intentional bias. The dairy industry holds a high economic influence in the development of milk studies. However, it seems that studies that do not declare conflict of interest can´t associate dairy products intake with a lower risk of osteoporosis and hip fracture. **[13]**

On the other hand, populations that are not deficient in calcium or vitamin D, supplementation has no influence. The preventive prescription of supplements could even be harmful. **[14]** In several studies, when the subjects took calcium supplements, their risk of hip fractures was not reduced, but may have been increased. In fact, randomized controlled trials suggest a 64% increased risk of hip fractures with calcium supplementation compared to the placebo pill.**[15]**

In the future, this model in which it is proposed to compare x-ray images can be replaced with the use of **Quantitative ultrasound (QUS)**. The validity in musculoskeletal ultrasound is well proven and technological advances in recent years have made both the devices and the images substantially improve. **[16]** Ultrasound has many advantages over radiographs since it is a cheaper, more available, more accessible, more innocuous and dynamic imaging technique.

Finally, it should be noted that this model **will not be valid for the use with patients who are receiving therapies that could reverse bone loss**, such as hormone and anabolic therapies. The EMD measures the difference of images. The algoritm is not able to differentiate if the image changes have a positive or negative interpretation.

### 5. Bibliography

*1. Vestergaard P, Rejnmark L: Increased mortality in patients with a hip fracture-effect of pre-morbid conditions and postfracture complication. Osteoporos Int. 2007; 18: 1583-93.*

*2. Brauer CA, Coca-Perraillon M, Cutler DM, Rosen AB: Incidence and mortality of hip fractures in the United States. JAMA. 2009; 302(14): 1573-9.*

*3. Watts NB, Ettinger B, LeBoff MS. FRAX facts. J Bone Miner Res. 2009;76:1–3*

*4. Melissa Premaor, Richard A. Parker, Steve Cummings. Predictive value of FRAX® for fracture in obese older women.J Bone Miner Res. 2013 Jan; 28(1): 188–195.7*

*5. J.A. Kanis, H. Johansson, A. Oden. A family history of fracture and fracture risk: a meta-analysis. Bone. 2004,35: 1029-1037.*

*6. Kenneth D. and Robert C. A Meta-Analysis of the Effects of Cigarette Smoking on Bone Mineral Density. Calcif Tissue Int. 2001 May; 68(5): 259–270.*

*7. Compston, J. Management of glucocorticoid-induced osteoporosis. Nat Rev Rheumatol 6, 82–88 (2010).*

*8. J. A. Kanis et al., “A meta-analysis of prior corticosteroid use and fracture risk,” J. Bone Miner. Res., vol. 19, no. 6, pp. 893–899, 2004, doi: 10.1359/JBMR.040134.*

*9. Haugeberg, G., Uhlig, T., Falch.  Bone mineral density and frequency of osteoporosis in female patients with 
rheumatoid arthritis: Results from 394 patients in the Oslo County rheumatoid arthritis register. Arthritis & Rheumatism, 43: 522-530.(2000).*   

*10. Godos J, Giampieri F, Chisari E. Alcohol Consumption, Bone Mineral Density, and Risk of 
Osteoporotic Fractures: A Dose-Response Meta-Analysis. Int J Environ Res Public Health. 2022 Jan 28;19(3):1515.*

*11. C. I. Sánchez Montero, “Osteoporosis y diagnóstico por radiografía convencional de fracturas vertebrales osteoporóticas,” Rev. Medica Sinerg., vol. 3, no. 10, pp. 7–11, Sep. 2018*

*12. Hong AR, Kim SW. Effects of Resistance Exercise on Bone Health. Endocrinol Metab (Seoul). 2018 Dec;33(4):435-444. doi: 10.3803/EnM.2018.33.4.435. PMID: 30513557; PMCID: PMC6279907.*

*13. Malmir H, Larijani B, Esmaillzadeh A. Consumption of milk and dairy products and risk of osteoporosis and hip fracture: a systematic review and Meta-analysis. Crit Rev Food Sci Nutr. 2020;60(10):1722-1737. doi: 10.1080/10408398.2019.1590800. Epub 2019 Mar 26. PMID: 30909722.*

*14. Reid IR, Bolland MJ. Calcium and/or Vitamin D Supplementation for the Prevention of Fragility Fractures: Who Needs It? Nutrients. 2020 Apr 7;12(4):1011. doi: 10.3390/nu12041011. PMID: 32272593; PMCID: PMC7231370.*

*15. Bischoff-Ferrari HA, Dawson-Hughes B, Baron JA, et al. Calcium intake and hip fracture risk in men and women: a meta-analysis of prospective cohort studies and randomized controlled trials. 2007.*

*16. Njeh CF, Boivin CM, Langton CM. The role of ultrasound in the assessment of osteoporosis: a review. Osteoporos Int. 1997;7(1):7-22. doi: 10.1007/BF01623454. PMID: 9102067.*